#### Given that we calculate discounts on a per transaction and per sku basis, all we need to do is to sum up the sales and discounts to calculate the discount rate. 

#### Since the total sales per SKU per transaction can not be negative. With that in mind, we can just filter out these rows, and the rest, just keep them there as we have no reference as to whether a steep discount rate such as 85% is valid. For now, let's assume they are.


In [240]:
import pandas as pd
data_type = pd.read_csv('data_type.csv')

In [215]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [221]:
data_type.fillna('',inplace=True)

In [192]:
data_type.head()

,StoreID,TransactionID,Line,Sales,Count,Date,SKU,Type
0,A29,11-7270-00007-00057,36,850.0,1.0,3/11/12 00:00,A21773,
1,A29,11-7270-00007-00057,39,1500.0,1.0,3/11/12 00:00,A18009,
2,A29,11-7270-00007-00057,40,1500.0,1.0,3/11/12 00:00,A25592,
3,A29,11-7270-00007-00057,41,1600.0,1.0,3/11/12 00:00,A1449,
4,A29,11-7270-00007-00057,42,1600.0,1.0,3/11/12 00:00,A1449,


In [81]:
data_type[data_type.TransactionID=='11-7270-00007-00110'][-4:]

,StoreID,TransactionID,Line,Sales,Count,Date,SKU,Type
1152,A29,11-7270-00007-00110,26,2195.0,1.0,3/11/12 00:00,A6007,
1153,A29,11-7270-00007-00110,27,-845.0,0.0,3/11/12 00:00,A6007,
1154,A29,11-7270-00007-00110,28,-2195.0,-1.0,3/11/12 00:00,A6007,Regular_Return
1155,A29,11-7270-00007-00110,29,845.0,0.0,3/11/12 00:00,A6007,CreditBack_RegularReturn


In [82]:
data_type[(data_type.TransactionID=='12-7081-00015-00118')&(data_type.SKU=='A5483')]

,StoreID,TransactionID,Line,Sales,Count,Date,SKU,Type
482509,A33,12-7081-00015-00118,9,6495.0,1.0,3/12/12 00:00,A5483,


In [85]:
q = '''
    select
        a.StoreID, a.TransactionID, a.SKU, abs(b.Total_Sales)/a.Total_Sales as Discount_Rate
    from
    (select StoreID, TransactionID, SKU, sum(Sales) as Total_Sales from data_type where Count!=0
    group by TransactionID, SKU) a
    join
    (select StoreID, TransactionID, SKU, sum(Sales) as Total_Sales from data_type where Count=0 
     group by TransactionID, SKU) b
    on
    a.TransactionID=b.TransactionID and a.SKU=b.SKU
    '''
DiscountRate = pysqldf(q)

In [193]:
DiscountRate.head()

,StoreID,TransactionID,SKU,Discount_Rate
0,A60,11-0160-00019-00003,A10809,0.30
1,A60,11-0160-00019-00007,A10809,0.30
2,A60,11-0160-00019-00007,A15452,0.75
3,A60,11-0160-00019-00008,A10809,0.30
4,A60,11-0160-00019-00008,A1165,0.70


In [252]:
len(DiscountRate.SKU.unique())

3298

In [91]:
DiscountRate.describe()

,Discount_Rate
count,104259.000000
mean,2.099115
std,266.821508
min,0.000133
25%,0.168074
50%,0.275766
75%,0.350046
max,44469.149528


#### Tukey IQR

In [92]:
# Check outliers as a reference
import numpy as np
def find_outliers_tukey(x):
    q1 = np.percentile(x,25)
    q3 = np.percentile(x,75)
    iqr = q3 - q1
    floor = q1 - 1.5*iqr
    ceiling = q3 + 1.5*iqr
    outlier_indices = list(x.index[(x < floor) | (x > ceiling)])
    outlier_values = list(x[outlier_indices])
    
    return outlier_indices, outlier_values

In [94]:
np.set_printoptions(formatter={'float_kind':lambda x: "%.2f" % x})
outlier_var = ['Discount_Rate']
for x in outlier_var:
    tukey_indices, tukey_values = find_outliers_tukey(DiscountRate[x])
    a = np.sort(tukey_values)
    print "Feature '{0}' first and last 10 outliers：{1}{2}\n".format(x,np.unique(a)[:10],np.unique(a)[-10:])
    print "Number of outliers: {0}\n".format(len(tukey_values))

Feature 'Discount_Rate' first and last 10 outliers：[0.62 0.62 0.62 0.62 0.63 0.63 0.63 0.63 0.64 0.64][0.99 1.06 1.12 1.23 1.27 26923.08 38910.51 39393.94 40677.97 44469.15]

Number of outliers: 1710



In [102]:
df = DiscountRate.iloc[tukey_indices]

In [154]:
df[(df.Discount_Rate>=0.85)].shape

(26, 4)

#### Discount_Rate>1000%

In [125]:
df[df.Discount_Rate>10]

,StoreID,TransactionID,SKU,Discount_Rate
4123,A78,11-5634-00008-00182,A23790,44469.149528
24226,A35,12-2581-00008-00083,A24995,26923.076923
46350,A78,12-5635-00018-00060,A23790,38910.505837
60089,A14,12-7016-00009-00039,A14624,40677.966102
62664,A13,12-7050-00013-00017,A25852,39393.939394


In [120]:
data_type[(data_type.TransactionID=='11-5634-00008-00182')&(data_type.SKU=='A23790')]

,StoreID,TransactionID,Line,Sales,Count,Date,SKU,Type
30407,A78,11-5634-00008-00182,8,1.799,1.0,3/11/12 00:00,A23790,
30408,A78,11-5634-00008-00182,9,-80000.000,0.0,3/11/12 00:00,A23790,


In [126]:
data_type[(data_type.TransactionID=='12-2581-00008-00083')&(data_type.SKU=='A24995')]

,StoreID,TransactionID,Line,Sales,Count,Date,SKU,Type
497375,A35,12-2581-00008-00083,3,1.56,24.0,3/12/12 00:00,A24995,
497376,A35,12-2581-00008-00083,4,-42000.00,0.0,3/12/12 00:00,A24995,


#### 100% <= Discount_Rate <= 1000%


In [135]:
df[(df.Discount_Rate>=1) &(df.Discount_Rate<=10)]

,StoreID,TransactionID,SKU,Discount_Rate
52374,A72,12-6643-00005-00370,A12887,1.265442
55653,A26,12-6861-00012-00649,A21659,1.230473
58895,A14,12-7016-00001-00350,A22127,1.061914
66083,A27,12-7070-00007-00002,A13772,1.115240


In [121]:
#120.6%
data_type[(data_type.TransactionID=='12-6643-00005-00370')&(data_type.SKU=='A12887')]

,StoreID,TransactionID,Line,Sales,Count,Date,SKU,Type
889800,A72,12-6643-00005-00370,12,2995.0,1.0,3/12/12 00:00,A12887,
889801,A72,12-6643-00005-00370,13,-3790.0,0.0,3/12/12 00:00,A12887,


In [122]:
#123%
data_type[(data_type.TransactionID=='12-6861-00012-00649')&(data_type.SKU=='A21659')]

,StoreID,TransactionID,Line,Sales,Count,Date,SKU,Type
365123,A26,12-6861-00012-00649,8,12995.0,1.0,3/12/12 00:00,A21659,
365124,A26,12-6861-00012-00649,9,-7995.0,0.0,3/12/12 00:00,A21659,
365126,A26,12-6861-00012-00649,12,-7995.0,0.0,3/12/12 00:00,A21659,


#### 85% < Discount_Rate < 100%


In [136]:
df[(df.Discount_Rate>0.85) &(df.Discount_Rate<1)].head()

,StoreID,TransactionID,SKU,Discount_Rate
18977,A51,12-1803-00009-00330,A23876,0.988467
19021,A51,12-1803-00013-00363,A21325,0.887387
19022,A51,12-1803-00013-00364,A21325,0.887383
19023,A51,12-1803-00013-00364,A25771,0.887387
34681,A65,12-3572-00001-00398,A6483,0.896104


In [132]:
df[(df.Discount_Rate>0.85) &(df.Discount_Rate<1)].shape

(17, 4)

In [138]:
#88.7%
data_type[(data_type.TransactionID=='12-1803-00013-00363')&(data_type.SKU=='A21325')]

,StoreID,TransactionID,Line,Sales,Count,Date,SKU,Type
705367,A51,12-1803-00013-00363,1,1.332,72.0,3/12/12 00:00,A21325,
705368,A51,12-1803-00013-00363,2,-1.182,0.0,3/12/12 00:00,A21325,


In [140]:
#88.7%
data_type[(data_type.TransactionID=='12-1803-00013-00364')&(data_type.SKU=='A25771')]

,StoreID,TransactionID,Line,Sales,Count,Date,SKU,Type
705373,A51,12-1803-00013-00364,7,44400.0,24.0,3/12/12 00:00,A25771,
705374,A51,12-1803-00013-00364,8,-39400.0,0.0,3/12/12 00:00,A25771,


In [139]:
#88.7%
data_type[(data_type.TransactionID=='12-1803-00013-00364')&(data_type.SKU=='A21325')]

,StoreID,TransactionID,Line,Sales,Count,Date,SKU,Type
705369,A51,12-1803-00013-00364,1,1.29500,70.0,3/12/12 00:00,A21325,
705370,A51,12-1803-00013-00364,2,-1.14917,0.0,3/12/12 00:00,A21325,
705371,A51,12-1803-00013-00364,4,81400.00000,44.0,3/12/12 00:00,A21325,
705372,A51,12-1803-00013-00364,5,-72233.00000,0.0,3/12/12 00:00,A21325,


In [142]:
#89.6%
data_type[(data_type.TransactionID=='12-3572-00001-00398')&(data_type.SKU=='A6483')][:4]

,StoreID,TransactionID,Line,Sales,Count,Date,SKU,Type
815391,A65,12-3572-00001-00398,4,1925.0,1.0,3/12/12 00:00,A6483,
815392,A65,12-3572-00001-00398,5,-1725.0,0.0,3/12/12 00:00,A6483,
815393,A65,12-3572-00001-00398,6,1925.0,1.0,3/12/12 00:00,A6483,
815394,A65,12-3572-00001-00398,7,-1725.0,0.0,3/12/12 00:00,A6483,


#### On a per transaction, per SKU basis, drop those rows where discount rate is >100% (Or total sales is negative)

In [170]:
df1 = df[(df.Discount_Rate>1)]

In [171]:
df1.shape

(9, 4)

In [172]:
df1

,StoreID,TransactionID,SKU,Discount_Rate
4123,A78,11-5634-00008-00182,A23790,44469.149528
24226,A35,12-2581-00008-00083,A24995,26923.076923
46350,A78,12-5635-00018-00060,A23790,38910.505837
52374,A72,12-6643-00005-00370,A12887,1.265442
55653,A26,12-6861-00012-00649,A21659,1.230473
58895,A14,12-7016-00001-00350,A22127,1.061914
60089,A14,12-7016-00009-00039,A14624,40677.966102
62664,A13,12-7050-00013-00017,A25852,39393.939394
66083,A27,12-7070-00007-00002,A13772,1.115240


In [241]:
q = '''
    select a.*
    from data_type a
    left join df1 b
    on a.TransactionID=b.TransactionID and a.SKU=b.SKU
    where b.TransactionID is null
    '''
data_type = pysqldf(q)

In [242]:
data_type.shape

(1041677, 8)

<br>

#### Drop rows where total sales<0 on a per transaction, per SKU basis

In [243]:
df2 = data_type.groupby(['TransactionID','SKU']).sum()

In [244]:
df3 = df2[df2.Sales<0]

In [245]:
df3.head()

,,Line,Sales,Count
TransactionID,SKU,,,
11-7270-00008-00100,A17972,164,-1992.0,1.0
12-0033-00008-00220,A12434,31,-600.0,0.0
12-0033-00012-00071,A2341,21,-100.0,1.0
12-0630-00020-00361,A25914,6,-10000.0,-1.0
12-1755-00003-00078,A2684,34,-8995.0,0.0


In [246]:
df3.shape

(19, 3)

In [247]:
#Number of returns is larger than number of sales on a per transaction and per SKU basis
data_type[(data_type.TransactionID=='12-1755-00003-00078')&(data_type.SKU=='A2684')]

,StoreID,TransactionID,Line,Sales,Count,Date,SKU,Type
730466,A53,12-1755-00003-00078,5,9995.0,1.0,3/12/12 00:00,A2684,None
730467,A53,12-1755-00003-00078,6,-9995.0,-1.0,3/12/12 00:00,A2684,Regular_Return
730471,A53,12-1755-00003-00078,11,-9995.0,-1.0,3/12/12 00:00,A2684,Regular_Return
730472,A53,12-1755-00003-00078,12,1000.0,1.0,3/12/12 00:00,A2684,None


In [248]:
data_type[(data_type.TransactionID=='12-2581-00009-00066')&(data_type.SKU=='A21452')]

,StoreID,TransactionID,Line,Sales,Count,Date,SKU,Type
499732,A35,12-2581-00009-00066,12,3295.0,1.0,3/12/12 00:00,A21452,None
499738,A35,12-2581-00009-00066,21,-3295.0,-1.0,3/12/12 00:00,A21452,Regular_Return
499739,A35,12-2581-00009-00066,22,1666.0,1.0,3/12/12 00:00,A21452,None
499740,A35,12-2581-00009-00066,24,-3295.0,-1.0,3/12/12 00:00,A21452,Regular_Return
499741,A35,12-2581-00009-00066,25,1666.0,1.0,3/12/12 00:00,A21452,None
499742,A35,12-2581-00009-00066,27,-3295.0,-1.0,3/12/12 00:00,A21452,Regular_Return
499743,A35,12-2581-00009-00066,28,1666.0,1.0,3/12/12 00:00,A21452,None


**Treatment:** It's not wise to change those regular returns to orphan returns manually in this case, so I decide to drop those rows when total sales$<0$ on a per transaction and per SKU basis.


In [249]:
q = '''
    select a.*
    from data_type a
    left join df3 b
    on a.TransactionID=b.TransactionID and a.SKU=b.SKU
    where b.TransactionID is null
    '''
data_type = pysqldf(q)

In [250]:
data_type.shape

(1041577, 8)

In [251]:
data_type.to_csv('data_type.csv', sep=',',index=False)

<br><br>